In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# تنظیم دستگاه (GPU اگر در دسترس باشد، در غیر این صورت CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# مسیر داده‌ها
MELBOURNE_DATASET = "D:/M.A/T2/Deep Learning/Assignments/HM2 - Es.Kiani - 40311614/RBF/daily-minimum-temperatures-in-melbourne.csv"

# توابع فعال‌سازی
def sigmoid(x):
    return torch.sigmoid(x)

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def rbf(x, center, sigma=1.0):
    return torch.exp(-torch.sum((x - center) ** 2, dim=-1) / (2 * sigma ** 2))

def rbf_derivative(x, center, sigma=1.0):
    rbf_val = rbf(x, center, sigma)
    return -rbf_val * (x - center) / (sigma ** 2)

def leaky_relu(x, alpha=0.1):
    return torch.where(x > 0, x, alpha * x)

def leaky_relu_derivative(x, alpha=0.1):
    return torch.where(x > 0, torch.ones_like(x), alpha * torch.ones_like(x))

# آماده‌سازی سری زمانی
def create_time_series(data, num_inputs, steps_ahead=3):
    X = torch.zeros((data.shape[0] - num_inputs - steps_ahead + 1, num_inputs))
    Y = torch.zeros((data.shape[0] - num_inputs - steps_ahead + 1))
    for i in range(X.shape[0]):
        X[i] = data[i:i + num_inputs]
        Y[i] = data[i + num_inputs + steps_ahead - 1]
    return X, Y

# بارگذاری و نرمال‌سازی داده‌ها
def load_and_normalize_data(file_path):
    data = pd.read_csv(file_path)
    temperatures = torch.tensor(data['Daily minimum temperatures in Melbourne, Australia, 1981-1990'].to_numpy(), dtype=torch.float32)
    min_val, max_val = torch.min(temperatures), torch.max(temperatures)
    normalized_data = (temperatures - min_val) / (max_val - min_val)
    return normalized_data, min_val, max_val

# تقسیم داده‌ها به train/test/val
def split_data(X, Y, train_ratio=0.7, val_ratio=0.1):
    num_data = X.shape[0]
    num_train = int(num_data * train_ratio)
    num_val = int(num_data * val_ratio)
    indices = torch.randperm(num_data)
    X, Y = X[indices], Y[indices]
    X_train, Y_train = X[:num_train], Y[:num_train]
    X_val, Y_val = X[num_train:num_train + num_val], Y[num_train:num_train + num_val]
    X_test, Y_test = X[num_train + num_val:], Y[num_train + num_val:]
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

# هایپرپارامترها
num_inputs = 30
n1_rbf = 20
n2 = 15
n3 = 10
n1_perceptron = 5
n2_perceptron = 1
eta_rbf = 0.25
eta_auto = 0.25
eta_perceptron = 0.25
epochs_auto = 50
epochs_perceptron = 50
sigma_rbf = 0.1

# بارگذاری داده‌ها
data, min_val, max_val = load_and_normalize_data(MELBOURNE_DATASET)
X, Y = create_time_series(data, num_inputs, steps_ahead=3)
X_train, Y_train, X_val, Y_val, X_test, Y_test = split_data(X, Y)

# انتقال داده‌ها به GPU
X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

# مقداردهی اولیه
centers_enc = X_train[torch.randperm(X_train.shape[0])[:n1_rbf]].clone().detach().to(device)  # (20, 30)
w2_e = torch.rand(n2, n1_rbf, device=device) * 2 - 1  # (15, 20)
w3_e = torch.rand(n3, n2, device=device) * 2 - 1      # (10, 15)
w3_d = torch.rand(n2, n3, device=device) * 2 - 1      # (15, 10)
w2_d = torch.rand(num_inputs, n2, device=device) * 2 - 1  # (30, 15)
w1_p = torch.rand(n1_perceptron, n3, device=device) * 2 - 1  # (5, 10)
w2_p = torch.rand(n2_perceptron, n1_perceptron, device=device) * 2 - 1  # (1, 5)

# فعال کردن گرادیان برای وزن‌ها
w2_e.requires_grad_(True)
w3_e.requires_grad_(True)
w3_d.requires_grad_(True)
w2_d.requires_grad_(True)
w1_p.requires_grad_(True)
w2_p.requires_grad_(True)

# تعریف توابع کمکی برای پیش‌بینی
def compute_encoder_output(X):
    rbf_layer = torch.stack([rbf(X, c, sigma_rbf) for c in centers_enc], dim=0).T  # (batch, 20)
    net2_e = w2_e @ rbf_layer  # (15, 20) @ (20, batch) → (15, batch)
    o2_e = leaky_relu(net2_e)  # (15, batch)
    net3_e = w3_e @ o2_e       # (10, 15) @ (15, batch) → (10, batch)
    o3_e = leaky_relu(net3_e)  # (10, batch)
    return o3_e

def compute_perceptron_output(autoencoder_output):
    net1_p = w1_p @ autoencoder_output  # (5, 10) @ (10,) → (5,)
    o1_p = sigmoid(net1_p)              # (5,)
    net2_p = w2_p @ o1_p                # (1, 5) @ (5,) → (1,)
    o2_p = sigmoid(net2_p)              # (1,)
    return o2_p

# آموزش Autoencoder
for t in range(epochs_auto):
    for i in range(X_train.shape[0]):
        X = X_train[i].unsqueeze(0)  # (1, 30)

        # Feedforward - Encoder
        o1_e = torch.stack([rbf(X, c, sigma_rbf) for c in centers_enc], dim=0)  # (20,)
        net2_e = w2_e @ o1_e  # (15, 20) @ (20,) → (15,)
        o2_e = leaky_relu(net2_e)  # (15,)
        net3_e = w3_e @ o2_e  # (10, 15) @ (15,) → (10,)
        o3_e = leaky_relu(net3_e)  # (10,)

        # Decoder
        net3_d = w3_d @ o3_e  # (15, 10) @ (10,) → (15,)
        o3_d = leaky_relu(net3_d)  # (15,)
        net2_d = w2_d @ o3_d  # (30, 15) @ (15,) → (30,)
        o1_d = sigmoid(net2_d)  # (30,)

        # Backpropagation
        loss = torch.mean((X.squeeze(0) - o1_d) ** 2)  # (30,) - (30,)
        loss.backward()

        with torch.no_grad():
            # به‌روزرسانی وزن‌ها
            w2_e -= eta_auto * w2_e.grad
            w3_e -= eta_auto * w3_e.grad
            w3_d -= eta_auto * w3_d.grad
            w2_d -= eta_auto * w2_d.grad

            # به‌روزرسانی مراکز RBF
            for j in range(n1_rbf):
                grad_center_enc = rbf_derivative(X, centers_enc[j], sigma_rbf).squeeze(0)  # (30,)
                delta = torch.sum(w2_e.grad[:, j])  # Scalar from (15,)
                centers_enc[j] += eta_rbf * grad_center_enc * delta  # (30,) * scalar

            # پاک کردن گرادیان‌ها
            w2_e.grad.zero_()
            w3_e.grad.zero_()
            w3_d.grad.zero_()
            w2_d.grad.zero_()

    if t % 10 == 0:
        print(f"Epoch {t}")

# آموزش Perceptron
mse_train = torch.zeros(epochs_perceptron, device=device)
mse_test = torch.zeros(epochs_perceptron, device=device)

for t in range(epochs_perceptron):
    for i in range(X_train.shape[0]):
        autoencoder_out = compute_encoder_output(X_train[i].unsqueeze(0)).squeeze()  # (10,)
        
        # Forward pass - Perceptron
        net1_p = w1_p @ autoencoder_out  # (5, 10) @ (10,) → (5,)
        o1_p = sigmoid(net1_p)           # (5,)
        net2_p = w2_p @ o1_p             # (1, 5) @ (5,) → (1,)
        o2_p = sigmoid(net2_p)           # (1,)

        # Backpropagation
        loss = (Y_train[i] - o2_p) ** 2
        loss.backward()

        with torch.no_grad():
            w2_p -= eta_perceptron * w2_p.grad
            w1_p -= eta_perceptron * w1_p.grad
            w2_p.grad.zero_()
            w1_p.grad.zero_()

    # پیش‌بینی‌ها
    with torch.no_grad():
        predictions_train = torch.stack([compute_perceptron_output(compute_encoder_output(x.unsqueeze(0)).squeeze()) for x in X_train]).squeeze()
        predictions_test = torch.stack([compute_perceptron_output(compute_encoder_output(x.unsqueeze(0)).squeeze()) for x in X_test]).squeeze()
        mse_train[t] = torch.mean((Y_train - predictions_train) ** 2)
        mse_test[t] = torch.mean((Y_test - predictions_test) ** 2)

    if t % 10 == 0:
        print(f"Epoch {t}")

# انتقال داده‌ها به CPU برای رسم و محاسبات نهایی
predictions_train = predictions_train.cpu().numpy()
predictions_test = predictions_test.cpu().numpy()
Y_train = Y_train.cpu().numpy()
Y_test = Y_test.cpu().numpy()
mse_train = mse_train.cpu().numpy()
mse_test = mse_test.cpu().numpy()

# نرمال‌زدایی معکوس
predictions_train_denorm = predictions_train * (max_val - min_val).item() + min_val.item()
predictions_test_denorm = predictions_test * (max_val - min_val).item() + min_val.item()
Y_train_denorm = Y_train * (max_val - min_val).item() + min_val.item()
Y_test_denorm = Y_test * (max_val - min_val).item() + min_val.item()

# رسم نتایج
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.plot(Y_train_denorm[:100], label="Actual", color="blue", alpha=0.7)
plt.plot(predictions_train_denorm[:100], label="Predicted", color="red", alpha=0.7)
plt.title("Train Data: Actual vs Predicted (Denormalized)")
plt.xlabel("Sample Index")
plt.ylabel("Temperature (°C)")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(Y_test_denorm[:100], label="Actual", color="blue", alpha=0.7)
plt.plot(predictions_test_denorm[:100], label="Predicted", color="red", alpha=0.7)
plt.title("Test Data: Actual vs Predicted (Denormalized)")
plt.xlabel("Sample Index")
plt.ylabel("Temperature (°C)")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(mse_train, label="MSE Train", color="blue")
plt.plot(mse_test, label="MSE Test", color="red")
plt.title("MSE over Epochs")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()

plt.subplot(2, 2, 4)
plt.scatter(Y_test_denorm, predictions_test_denorm, color="purple", alpha=0.5)
plt.plot([Y_test_denorm.min(), Y_test_denorm.max()], [Y_test_denorm.min(), Y_test_denorm.max()], 'k--', lw=2)
plt.title("Regression: Actual vs Predicted (Test Data)")
plt.xlabel("Actual Temperature (°C)")
plt.ylabel("Predicted Temperature (°C)")

plt.tight_layout()
plt.show()

# محاسبه MAE
def calculate_mae(actual, predicted):
    return np.mean(np.abs(actual - predicted))

mae_train = calculate_mae(Y_train, predictions_train)
mae_test = calculate_mae(Y_test, predictions_test)
mae_train_denorm = calculate_mae(Y_train_denorm, predictions_train_denorm)
mae_test_denorm = calculate_mae(Y_test_denorm, predictions_test_denorm)

print("Performance Metrics (Normalized):")
print(f"Train MSE: {mse_train[-1]:.6f}")
print(f"Test MSE: {mse_test[-1]:.6f}")
print(f"Train MAE: {mae_train:.6f}")
print(f"Test MAE: {mae_test:.6f}")

print("\nPerformance Metrics (Denormalized):")
print(f"Train MSE (Denorm): {(mse_train[-1] * (max_val - min_val)**2).item():.6f}")
print(f"Test MSE (Denorm): {(mse_test[-1] * (max_val - min_val)**2).item():.6f}")
print(f"Train MAE (Denorm): {mae_train_denorm:.6f}")
print(f"Test MAE (Denorm): {mae_test_denorm:.6f}")

Using device: cuda
Epoch 0
